In [1]:
import pandas as pd
import os
from datetime import datetime
import json
import re

In [2]:
def remove_urls (vTEXT):
    results = re.compile(r'http://[a-zA-Z0-9.?/&=:]*', re.S)
    dd = results.sub("", vTEXT)
    return dd

def remove_urls_https (vTEXT):
    results = re.compile(r'https://[a-zA-Z0-9.?/&=:]*', re.S)
    dd = results.sub("", vTEXT)
    return dd

def remove_incomplete_urls (vTEXT):
    vTEXT = re.sub(r'http://…', '', vTEXT, flags=re.MULTILINE)
    return(vTEXT)

def remove_double_line (vTEXT):
    vTEXT = re.sub(r'--', '', vTEXT, flags=re.MULTILINE)
    return(vTEXT)

def remove_RT_at (vTEXT):
    vTEXT = re.sub(r'(RT @)(.*)(\:)', "", vTEXT, flags=re.MULTILINE)
    return(vTEXT)

def remove_via_at (vTEXT):
    vTEXT = re.sub(r'(via @)(.*)(\.)', "", vTEXT, flags=re.MULTILINE)
    return(vTEXT)

def remove_Ellipsis (vTEXT):
    vTEXT = re.sub(r'  …', "", vTEXT, flags=re.MULTILINE)
    return(vTEXT)

In [3]:
# const
DATA_PATH = "../data"
PRICE_PATH = DATA_PATH + "/price/"
TWEET_PATH = DATA_PATH + "/tweet/"
TSV_PATH = DATA_PATH + "/tsv/"

In [4]:
# input
stock_name = input("Input the stock name of data: ")
delay = int(input("Input delay (number of day): "))

Input the stock name of data: AAPL
Input delay (number of day): 1


In [5]:
# read price csv data
price_df = pd.read_csv(PRICE_PATH + stock_name + ".csv")
price_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2012-09-04,95.108574,96.448570,94.928574,96.424286,87.121140,91973000
1,2012-09-05,96.510002,96.621429,95.657143,95.747147,86.509338,84093800
2,2012-09-06,96.167145,96.898575,95.828575,96.610001,87.288956,97799100
3,2012-09-07,96.864288,97.497147,96.538574,97.205711,87.827171,82416600
4,2012-09-10,97.207146,97.612854,94.585716,94.677139,85.542564,121999500


In [6]:
# add label column to price_df
price_df['Label'] = price_df.apply(lambda row: 1 if row.Close - row.Open >= 0 else 0, axis = 1)
price_df.index = pd.to_datetime(price_df['Date'], format = '%Y-%m-%d')
price_df['Date'] = pd.to_datetime(price_df['Date'], format = '%Y-%m-%d')
price_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Label
Date,,,,,,,,
2012-09-04,2012-09-04,95.108574,96.448570,94.928574,96.424286,87.121140,91973000,1
2012-09-05,2012-09-05,96.510002,96.621429,95.657143,95.747147,86.509338,84093800,0
2012-09-06,2012-09-06,96.167145,96.898575,95.828575,96.610001,87.288956,97799100,1
2012-09-07,2012-09-07,96.864288,97.497147,96.538574,97.205711,87.827171,82416600,1
2012-09-10,2012-09-10,97.207146,97.612854,94.585716,94.677139,85.542564,121999500,0


In [7]:
# Stage 1: convert eact tweet file with correct stock date (map off market day to market day)

# path name
dir_str = TWEET_PATH + stock_name
dir_obj = os.fsencode(TWEET_PATH + stock_name)

# dataframe
tweet_df = pd.DataFrame(columns=['date', 'id', 'label', 'alpha', 'text'])
current_date = None
index = 0

for i, filename_byte in enumerate(os.listdir(dir_obj)):
    filename = os.fsdecode(filename_byte)
    date = datetime.strptime(filename, '%Y-%m-%d')
    
    # find nearest market day after current_date in price_df
    temp_df = price_df.loc[price_df.index >= date]
    current_date = temp_df.index[temp_df.index.get_loc(current_date, method='nearest')]
    
    with open(dir_str + "/" + filename, 'r') as file:
        for line in file:
            json_str = json.loads(line)
            temp_text=json_str['text']
            temp_text = remove_urls(temp_text)
            temp_text = remove_urls_https(temp_text)
            temp_text = remove_incomplete_urls(temp_text)
            temp_text = remove_RT_at(temp_text)
            temp_text = remove_via_at(temp_text)
            temp_text=  remove_double_line(temp_text)
            temp_text=  remove_Ellipsis(temp_text)
            json_str['text']=temp_text
            tweet_df.loc[index] = {'date': current_date, 'id': 0, 'label': 0, 'alpha': 'a', 'text': json_str['text']}
            index += 1
            
    if i % 100 == 0:
        print("Finished processing %d file" % i)

Finished processing 0 file
Finished processing 100 file
Finished processing 200 file
Finished processing 300 file
Finished processing 400 file
Finished processing 500 file
Finished processing 600 file
Finished processing 700 file


In [8]:
# replace id column with index
tweet_df['id'] = tweet_df.index
tweet_df.head(10)

,date,id,label,alpha,text
0,2014-01-02,0,0,a,Summary of Yesterday's Webcast Featuring $AAP...
1,2014-01-02,1,0,a,Summary of Yesterday's Webcast Featuring $AAP...
2,2014-01-02,2,0,a,iTV Will Boost Apple $AAPL #APPLE
3,2014-01-02,3,0,a,#iPhone users are more intelligent than #Samsu...
4,2014-01-02,4,0,a,Summary of Yesterday's Webcast Featuring $AAP...
5,2014-01-02,5,0,a,2013 Wrap-Up And Trading Set Review - Part III...
6,2014-01-02,6,0,a,Apple Screwed Up Big Time $AMZN $AAPL
7,2014-01-02,7,0,a,Summary of Yesterday's Webcast Featuring $AAP...
8,2014-01-02,8,0,a,$AAPL - Wall St. kicks off New Year on lower n...
9,2014-01-02,9,0,a,Here's how Apple could be making a huge push ...


In [9]:
# Stage 2: add postive / negative label to tweet_df by price number
p_set = set(price_df['Date'].tolist())
t_set = set(tweet_df['date'].tolist())
market_days = list(p_set.intersection(t_set))
market_days.sort()

for i, day in enumerate(market_days):
    
    # end the loop if delay is over
    if i + delay > len(market_days) - 1:
        break
        
    delay_day = market_days[i + delay]
    label = price_df.loc[price_df.index == delay_day, 'Label'].values[0]
    tweet_df.loc[tweet_df['date'] == day, 'label'] = label

In [10]:
tweet_df[tweet_df['label'] == 1].head()

,date,id,label,alpha,text
41,2014-01-03,41,1,a,"Our Top 3 Trade Ideas for 2014 and Two $1,000..."
42,2014-01-03,42,1,a,The Five-Year-Old iPad: When Will People Stop ...
43,2014-01-03,43,1,a,"$AAPL was below this level two weeks ago, and ..."
44,2014-01-03,44,1,a,$AAPL Apple Sale: Best Buy Temporarily Cuts iP...
45,2014-01-03,45,1,a,Also added $AAPL common here w 541 stop


In [11]:
# Output to tsv format
tweet_df.to_csv(TSV_PATH + "aapl.tsv", sep=',', index=False)